# JSON形式

## JSON形式の特徴

## データサンプル

In [7]:
from urllib.request import urlopen
import os

# app_id = os.environ["ESTAT_APP_ID"]  # アプリケーションIDをOSの環境変数から取得

url = f"http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?appId=528ef50eee0854bda127ee70b47045020e73658d&lang=J&statsDataId=0003443838&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1&replaceSpChars=0"
with urlopen(url) as uf:
    with open("data/population-202302.json", "wb") as f:
        f.write(uf.read())

# JSON形式を扱うライブラリ

# jsonライブラリを使う

In [8]:
import json

with open("data/population-202302.json", "r") as f:
    data = json.load(f)

type(data)

dict

In [9]:
data

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0,
   'ERROR_MSG': '正常に終了しました。',
   'DATE': '2024-05-11T20:48:13.014+09:00'},
  'PARAMETER': {'LANG': 'J',
   'STATS_DATA_ID': '0003443838',
   'DATA_FORMAT': 'J',
   'START_POSITION': 1,
   'METAGET_FLG': 'Y',
   'EXPLANATION_GET_FLG': 'Y',
   'ANNOTATION_GET_FLG': 'Y',
   'REPLACE_SP_CHARS': 0,
   'CNT_GET_FLG': 'N',
   'SECTION_HEADER_FLG': 1},
  'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 2349,
    'FROM_NUMBER': 1,
    'TO_NUMBER': 2349},
   'TABLE_INF': {'@id': '0003443838',
    'STAT_NAME': {'@code': '00200524', '$': '人口推計'},
    'GOV_ORG': {'@code': '00200', '$': '総務省'},
    'STATISTICS_NAME': '人口推計 各月1日現在人口 概算値',
    'TITLE': {'@no': '001',
     '$': '概算値 年齢（５歳階級），男女，月別人口－総人口（各月１日現在）-令和２年国勢調査基準-'},
    'CYCLE': '月次',
    'SURVEY_DATE': 0,
    'OPEN_DATE': '2024-04-22',
    'SMALL_AREA': 0,
    'COLLECT_AREA': '全国',
    'MAIN_CATEGORY': {'@code': '02', '$': '人口・世帯'},
    'SUB_CATEGORY': {'@code': '01', '$': '人口'},
    'OVERALL_TO

In [10]:
data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]

{'TOTAL_NUMBER': 2349, 'FROM_NUMBER': 1, 'TO_NUMBER': 2349}

In [11]:
data["GET_STATS_DATA"]["STATISTICAL_DATA"]["TABLE_INF"]["TITLE"]

{'@no': '001', '$': '概算値 年齢（５歳階級），男女，月別人口－総人口（各月１日現在）-令和２年国勢調査基準-'}

In [12]:
data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"][0]

{'@tab': '001',
 '@cat01': '001',
 '@cat02': '000',
 '@cat03': '01000',
 '@cat04': '01',
 '@area': '00000',
 '@time': '2021001212',
 '@unit': '万人',
 '$': '12547'}

In [13]:
len(data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])

2349

In [14]:
values = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"]
for i, d in enumerate(values):
    for k, v in d.items():
        print(k, v, end="|")
    print()
    print("===")
    if i > 1:
        break

@tab 001|@cat01 001|@cat02 000|@cat03 01000|@cat04 01|@area 00000|@time 2021001212|@unit 万人|$ 12547|
===
@tab 001|@cat01 001|@cat02 000|@cat03 01000|@cat04 01|@area 00000|@time 2022000101|@unit 万人|$ 12544|
===
@tab 001|@cat01 001|@cat02 000|@cat03 01000|@cat04 01|@area 00000|@time 2022000202|@unit 万人|$ 12534|
===


## pandasのDataFrameに変換

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(
    data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"]
)
df.head()

,@tab,@cat01,@cat02,@cat03,@cat04,@area,@time,@unit,$
0,001,001,000,01000,01,00000,2021001212,万人,12547
1,001,001,000,01000,01,00000,2022000101,万人,12544
2,001,001,000,01000,01,00000,2022000202,万人,12534
3,001,001,000,01000,01,00000,2022000303,万人,12526
4,001,001,000,01000,01,00000,2022000404,万人,12519


In [17]:
df.shape

(2349, 9)

# pandasを使って読み込む

In [18]:
df_raw_json = pd.read_json("data/population-202302.json")
df_raw_json

,GET_STATS_DATA
RESULT,"{'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE..."
PARAMETER,"{'LANG': 'J', 'STATS_DATA_ID': '0003443838', '..."
STATISTICAL_DATA,"{'RESULT_INF': {'TOTAL_NUMBER': 2349, 'FROM_NU..."


## json_normalize関数を使う

In [19]:
import json

with open("data/population-202302.json", "r") as f:
    data = json.load(f)

values = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"]
df_norm_json = pd.json_normalize(values)

In [20]:
df_norm_json.head()

,@tab,@cat01,@cat02,@cat03,@cat04,@area,@time,@unit,$
0,001,001,000,01000,01,00000,2021001212,万人,12547
1,001,001,000,01000,01,00000,2022000101,万人,12544
2,001,001,000,01000,01,00000,2022000202,万人,12534
3,001,001,000,01000,01,00000,2022000303,万人,12526
4,001,001,000,01000,01,00000,2022000404,万人,12519


In [21]:
df_norm_json.shape

(2349, 9)

In [22]:
# カラム名に該当する情報をJSONから取得して、columnsに代入
columns = [
    d["@name"]
    for d in data["GET_STATS_DATA"]["STATISTICAL_DATA"]["CLASS_INF"][
        "CLASS_OBJ"
    ]
]

# JSONからの情報に不足している情報を付与
df_norm_json.columns = columns + ["単位", "人数"]
df_norm_json.head()

,表章項目,人口,男女別,年齢5歳階級,概算値,全国,時間軸（年月日現在）,単位,人数
0,001,001,000,01000,01,00000,2021001212,万人,12547
1,001,001,000,01000,01,00000,2022000101,万人,12544
2,001,001,000,01000,01,00000,2022000202,万人,12534
3,001,001,000,01000,01,00000,2022000303,万人,12526
4,001,001,000,01000,01,00000,2022000404,万人,12519
